You'll work with housing data for the city of Ames, Iowa, United States from 2006 to 2010. You can read more about why the data was collected here: https://www.tandfonline.com/doi/abs/10.1080/10691898.2011.11889627 

You can also read about the different columns in the data here: 

https://s3.amazonaws.com/dq-content/307/data_description.txt

In [1]:
import pandas as pd
pd.options.display.max_columns = 999
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import KFold

In [2]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
df.head()

Order        PID  MS SubClass MS Zoning  Lot Frontage  Lot Area Street  \
0      1  526301100           20        RL         141.0     31770   Pave   
1      2  526350040           20        RH          80.0     11622   Pave   
2      3  526351010           20        RL          81.0     14267   Pave   
3      4  526353030           20        RL          93.0     11160   Pave   
4      5  527105010           60        RL          74.0     13830   Pave   

  Alley Lot Shape Land Contour Utilities Lot Config Land Slope Neighborhood  \
0   NaN       IR1          Lvl    AllPub     Corner        Gtl        NAmes   
1   NaN       Reg          Lvl    AllPub     Inside        Gtl        NAmes   
2   NaN       IR1          Lvl    AllPub     Corner        Gtl        NAmes   
3   NaN       Reg          Lvl    AllPub     Corner        Gtl        NAmes   
4   NaN       IR1          Lvl    AllPub     Inside        Gtl      Gilbert   

  Condition 1 Condition 2 Bldg Type House Style  Overall Qual  Overall Cond  \
0        Norm        Norm      1Fam      1Story             6             5   
1       Feedr        Norm      1Fam      1Story             5             6   
2        Norm        Norm      1Fam      1Story             6             6   
3        Norm        Norm      1Fam      1Story             7             5   
4        Norm        Norm      1Fam      2Story             5             5   

   Year Built  Year Remod/Add Roof Style Roof Matl Exterior 1st Exterior 2nd  \
0        1960            1960        Hip   CompShg      BrkFace      Plywood   
1        1961            1961      Gable   CompShg      VinylSd      VinylSd   
2        1958            1958        Hip   CompShg      Wd Sdng      Wd Sdng   
3        1968            1968        Hip   CompShg      BrkFace      BrkFace   
4        1997            1998      Gable   CompShg      VinylSd      VinylSd   

  Mas Vnr Type  Mas Vnr Area Exter Qual Exter Cond Foundation Bsmt Qual  \
0        Stone         112.0         TA         TA     CBlock        TA   
1         None           0.0         TA         TA     CBlock        TA   
2      BrkFace         108.0         TA         TA     CBlock        TA   
3         None           0.0         Gd         TA     CBlock        TA   
4         None           0.0         TA         TA      PConc        Gd   

  Bsmt Cond Bsmt Exposure BsmtFin Type 1  BsmtFin SF 1 BsmtFin Type 2  \
0        Gd            Gd            BLQ         639.0            Unf   
1        TA            No            Rec         468.0            LwQ   
2        TA            No            ALQ         923.0            Unf   
3        TA            No            ALQ        1065.0            Unf   
4        TA            No            GLQ         791.0            Unf   

   BsmtFin SF 2  Bsmt Unf SF  Total Bsmt SF Heating Heating QC Central Air  \
0           0.0        441.0         1080.0    GasA         Fa           Y   
1         144.0        270.0          882.0    GasA         TA           Y   
2           0.0        406.0         1329.0    GasA         TA           Y   
3           0.0       1045.0         2110.0    GasA         Ex           Y   
4           0.0        137.0          928.0    GasA         Gd           Y   

  Electrical  1st Flr SF  2nd Flr SF  Low Qual Fin SF  Gr Liv Area  \
0      SBrkr        1656           0                0         1656   
1      SBrkr         896           0                0          896   
2      SBrkr        1329           0                0         1329   
3      SBrkr        2110           0                0         2110   
4      SBrkr         928         701                0         1629   

   Bsmt Full Bath  Bsmt Half Bath  Full Bath  Half Bath  Bedroom AbvGr  \
0             1.0             0.0          1          0              3   
1             0.0             0.0          1          0              2   
2             0.0             0.0          1          1              3   
3             1.0             0.0

In [3]:
def transform_features(df):
    return df


def select_features(df):
    return df[["Gr Liv Area", "SalePrice"]]


def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    
    numeric_train = train.select_dtypes(include=['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    
    # drop the target variable
    features = numeric_train.columns.drop("SalePrice")
    
    # train and predict
    lr = linear_model.LinearRegression()
    lr.fit(train[features], train["SalePrice"])
    predictions = lr.predict(test[features])
    
    # calculate the errors
    mse = mean_squared_error(test["SalePrice"], predictions)
    rmse = np.sqrt(mse)
    
    return rmse 

In [4]:
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse 

57088.25161263909

# Feature Engineering

In [5]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")

## Exploration

### Which columns contain less than 5% missing values?
- drop those that are above 5%
- fill nas of numeric columns with most popular values



In [6]:
rows_in_df = df.shape[0]
missing_values = df.isna().sum()
cols_to_drop = missing_values[(missing_values/rows_in_df*100)>5].sort_values()
cols_to_drop

Garage Type       157
Garage Yr Blt     159
Garage Finish     159
Garage Qual       159
Garage Cond       159
Lot Frontage      490
Fireplace Qu     1422
Fence            2358
Alley            2732
Misc Feature     2824
Pool QC          2917
dtype: int64

In [7]:
# drop those columns
df = df.drop(cols_to_drop.index, axis=1)

In [8]:
# get numeric columns whose missing values need to be handled
num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
fixable_numeric_cols = num_missing[((num_missing/rows_in_df*100)<=5) & (num_missing > 0)].sort_values()
fixable_numeric_cols

BsmtFin SF 1       1
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
Garage Cars        1
Garage Area        1
Bsmt Full Bath     2
Bsmt Half Bath     2
Mas Vnr Area      23
dtype: int64

In [9]:
# find most popular values for those columns where na's need to be filled
replacement_for_nas = df[fixable_numeric_cols.index].mode().loc[0,].to_dict()
replacement_for_nas

{'Bsmt Full Bath': 0.0,
 'Bsmt Half Bath': 0.0,
 'Bsmt Unf SF': 0.0,
 'BsmtFin SF 1': 0.0,
 'BsmtFin SF 2': 0.0,
 'Garage Area': 0.0,
 'Garage Cars': 2.0,
 'Mas Vnr Area': 0.0,
 'Total Bsmt SF': 0.0}

In [10]:
# replace missing values
df = df.fillna(replacement_for_nas)

In [11]:
# Verify that every numeric column has 0 missing values
df.select_dtypes(include=['int', 'float']).isnull().sum().sum()

0

### What new features can we create, that better capture the information in some of the features?
- Years Before Sale
- Years Since Remod

In [12]:
years_sold = df['Yr Sold'] - df['Year Built']
years_sold[years_sold < 0]

2180   -1
dtype: int64

In [13]:
years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
years_since_remod[years_since_remod < 0]

1702   -1
2180   -2
2181   -1
dtype: int64

In [14]:
# Create new columns
df['Years Before Sale'] = years_sold
df['Years Since Remod'] = years_since_remod

# Drop rows with negative values for both of these new features
df = df.drop([1702, 2180, 2181], axis=0)

# drop original year columns (not needed anymore)
df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)
df[['Years Before Sale','Years Since Remod','Yr Sold']].sample(5)

Years Before Sale  Years Since Remod  Yr Sold
1997                128                 20     2007
2016                 81                 57     2007
249                   2                  2     2010
2473                  1                  1     2006
296                  51                  8     2010

## update transform features with new insights/code

In [15]:
def transform_features(df):
    # drop numeric cols with more than 5% na's
    rows_in_df = df.shape[0]
    missing_values = df.isna().sum()
    cols_to_drop = missing_values[(missing_values/rows_in_df*100)>5].sort_values()
    df = df.drop(cols_to_drop.index, axis=1)
    
    # drop string cols with any na's
    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    string_cols_to_drop = text_mv_counts[text_mv_counts > 0]
    df = df.drop(string_cols_to_drop.index, axis=1)
    
    # replace numeric columns with missing values with most popular values
    num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
    fixable_numeric_cols = num_missing[((num_missing/rows_in_df*100)<=5) & (num_missing > 0)].sort_values()
    replacement_for_nas = df[fixable_numeric_cols.index].mode().loc[0,].to_dict()
    df = df.fillna(replacement_for_nas)
    
    # build new features
    years_sold = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod
    df = df.drop([1702, 2180, 2181], axis=0)
    
    # drop not necessary columns
    df = df.drop(["PID", "Order", "Mo Sold", "Sale Condition", "Sale Type", "Year Built", "Year Remod/Add"], axis=1)
    return df 

In [16]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse 

55275.36731241307

# Feature Selection

In [29]:
# checkout correlations with Sale Price
numerical_df = transform_df.select_dtypes(include=['int', 'float'])
correlations = numerical_df.corr()
abs_corr_coeffs = correlations["SalePrice"].abs().sort_values(ascending=False)
abs_corr_coeffs

SalePrice            1.000000
Overall Qual         0.801206
Gr Liv Area          0.717596
Garage Cars          0.648361
Total Bsmt SF        0.644012
Garage Area          0.641425
1st Flr SF           0.635185
Years Before Sale    0.558979
Full Bath            0.546118
Years Since Remod    0.534985
Mas Vnr Area         0.506983
TotRms AbvGrd        0.498574
Fireplaces           0.474831
BsmtFin SF 1         0.439284
Wood Deck SF         0.328183
Open Porch SF        0.316262
Half Bath            0.284871
Bsmt Full Bath       0.276258
2nd Flr SF           0.269601
Lot Area             0.267520
Bsmt Unf SF          0.182751
Bedroom AbvGr        0.143916
Enclosed Porch       0.128685
Kitchen AbvGr        0.119760
Screen Porch         0.112280
Overall Cond         0.101540
MS SubClass          0.085128
Pool Area            0.068438
Low Qual Fin SF      0.037629
Bsmt Half Bath       0.035875
3Ssn Porch           0.032268
Yr Sold              0.030358
Misc Val             0.019273
BsmtFin SF

In [30]:
# Drop columns with less than 0.4 correlation with SalePrice
transform_df = transform_df.drop(abs_corr_coeffs[abs_corr_coeffs < 0.4].index, axis=1)

In [31]:
# Create a list of column names from documentation that are *meant* to be categorical
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

In [32]:
# Which categorical columns have we still carried with us? We'll test these 
transform_cat_cols = []
for col in nominal_features:
    if col in transform_df.columns:
        transform_cat_cols.append(col)
transform_cat_cols

['MS Zoning',
 'Street',
 'Land Contour',
 'Lot Config',
 'Neighborhood',
 'Condition 1',
 'Condition 2',
 'Bldg Type',
 'House Style',
 'Roof Style',
 'Roof Matl',
 'Exterior 1st',
 'Exterior 2nd',
 'Foundation',
 'Heating',
 'Central Air']

In [37]:
# How many unique values in each categorical column?
uniqueness_counts = transform_df[transform_cat_cols].nunique()
uniqueness_counts

MS Zoning        7
Street           2
Land Contour     4
Lot Config       5
Neighborhood    28
Condition 1      9
Condition 2      8
Bldg Type        5
House Style      8
Roof Style       6
Roof Matl        8
Exterior 1st    16
Exterior 2nd    17
Foundation       6
Heating          6
Central Air      2
dtype: int64

In [38]:
# cutoff of 10 unique values
drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
transform_df = transform_df.drop(drop_nonuniq_cols, axis=1)

In [41]:
# Select just the remaining text columns and convert to categorical
text_cols = transform_df.select_dtypes(include=['object'])
for col in text_cols:
    transform_df[col] = transform_df[col].astype('category')
transform_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2927 entries, 0 to 2929
Data columns (total 36 columns):
MS Zoning            2927 non-null category
Street               2927 non-null category
Lot Shape            2927 non-null category
Land Contour         2927 non-null category
Utilities            2927 non-null category
Lot Config           2927 non-null category
Land Slope           2927 non-null category
Condition 1          2927 non-null category
Condition 2          2927 non-null category
Bldg Type            2927 non-null category
House Style          2927 non-null category
Overall Qual         2927 non-null int64
Roof Style           2927 non-null category
Roof Matl            2927 non-null category
Mas Vnr Area         2927 non-null float64
Exter Qual           2927 non-null category
Exter Cond           2927 non-null category
Foundation           2927 non-null category
BsmtFin SF 1         2927 non-null float64
Total Bsmt SF        2927 non-null float64
Heating            

In [42]:
# Create dummy columns and add back to the dataframe!
transform_df = pd.concat([
    transform_df, 
    pd.get_dummies(transform_df.select_dtypes(include=['category']))
], axis=1).drop(text_cols,axis=1)
transform_df.head()

MS Zoning Street Lot Shape Land Contour Utilities Lot Config Land Slope  \
0        RL   Pave       IR1          Lvl    AllPub     Corner        Gtl   
1        RH   Pave       Reg          Lvl    AllPub     Inside        Gtl   
2        RL   Pave       IR1          Lvl    AllPub     Corner        Gtl   
3        RL   Pave       Reg          Lvl    AllPub     Corner        Gtl   
4        RL   Pave       IR1          Lvl    AllPub     Inside        Gtl   

  Condition 1 Condition 2 Bldg Type House Style  Overall Qual Roof Style  \
0        Norm        Norm      1Fam      1Story             6        Hip   
1       Feedr        Norm      1Fam      1Story             5      Gable   
2        Norm        Norm      1Fam      1Story             6        Hip   
3        Norm        Norm      1Fam      1Story             7        Hip   
4        Norm        Norm      1Fam      2Story             5      Gable   

  Roof Matl  Mas Vnr Area Exter Qual Exter Cond Foundation  BsmtFin SF 1  \
0   CompShg         112.0         TA         TA     CBlock         639.0   
1   CompShg           0.0         TA         TA     CBlock         468.0   
2   CompShg         108.0         TA         TA     CBlock         923.0   
3   CompShg           0.0         Gd         TA     CBlock        1065.0   
4   CompShg           0.0         TA         TA      PConc         791.0   

   Total Bsmt SF Heating Heating QC Central Air  1st Flr SF  Gr Liv Area  \
0         1080.0    GasA         Fa           Y        1656         1656   
1          882.0    GasA         TA           Y         896          896   
2         1329.0    GasA         TA           Y        1329         1329   
3         2110.0    GasA         Ex           Y        2110         2110   
4          928.0    GasA         Gd           Y         928         1629   

   Full Bath Kitchen Qual  TotRms AbvGrd Functional  Fireplaces  Garage Cars  \
0          1           TA              7        Typ           2          2.0   
1          1           TA              5        Typ           0          1.0   
2          1           Gd              6        Typ           0          1.0   
3          2           Ex              8        Typ           2          2.0   
4          2           TA              6        Typ           1          2.0   

   Garage Area Paved Drive  SalePrice  Years Before Sale  Years Since Remod  \
0        528.0           P     215000                 50                 50   
1        730.0           Y     105000                 49                 49   
2        312.0           Y     172000                 52                 52   
3        522.0           Y     244000                 42                 42   
4        482.0           Y     189900                 13                 12   

   MS Zoning_A (agr)  MS Zoning_C (all)  MS Zoning_FV  MS Zoning_I (all)  \
0                  0                  0             0                  0   
1                  0                  0             0                  0   
2                  0                  0             0                  0   
3                  0                  0             0                  0   
4                  0                  0             0                  0   

   MS Zoning_RH  MS Zoning_RL  MS Zoning_RM  Street_Grvl  Street_Pave  \
0             0             1             0            0            1   
1             1             0             0            0            1   
2             0             1             0            0            1   
3             0             1             0            0            1   
4             0             1             0            0            1   

   Lot Shape_IR1  Lot Shape_IR2  Lot Shape_IR3  Lot Shape_Reg  \
0              1              0              0              0   
1              0              0              0              1   
2              1              0              0              0   
3              0              0              0          

In [43]:
# update select features function
def select_features(df, coeff_threshold=0.4, uniq_threshold=10):
    numerical_df = df.select_dtypes(include=['int', 'float'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)

    uniqueness_counts = df[transform_cat_cols].nunique().sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
    df = df.drop(drop_nonuniq_cols, axis=1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1).drop(text_cols,axis=1)
    
    return df

In [44]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse 

33367.287183402594

# Train and Test (with cv)

In [45]:
def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = linear_model.LinearRegression()
    
    if k == 0:
        train = df[:1460]
        test = df[1460:]

        lr.fit(train[features], train["SalePrice"])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], predictions)
        rmse = np.sqrt(mse)

        return rmse
    
    if k == 1:
        # Randomize *all* rows (frac=1) from `df` and return
        shuffled_df = df.sample(frac=1, )
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train["SalePrice"])
        predictions_one = lr.predict(test[features])        
        
        mse_one = mean_squared_error(test["SalePrice"], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(test[features], test["SalePrice"])
        predictions_two = lr.predict(train[features])        
       
        mse_two = mean_squared_error(train["SalePrice"], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse

In [46]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df, k=0)

rmse 

33367.287183402594

In [47]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df, k=5)

rmse 

[25811.25071425468, 37453.1524908695, 27606.641296716843, 26851.48369106887, 27337.444518478256]


29011.994542277633